# Iterative Blocked Householder QR Using TSQR Panel Factorization

In this notebook, we will introduce the mathematical background for an iterative blocked Householder QR algorithm that implements tall-skinny QR (TSQR) for panel factorization. We will also attempt to represent the algorithm using cascades of Einsums. While the algorithms here only work for real matrices, all concepts can be easily generalized to complex matrices using Hermitian transposes.

**This notebook is primarily based on the following papers:**
- [Reconstructing Householder Vectors from Tall-Skinny QR](https://ieeexplore.ieee.org/document/6877344)
- [High Performance Householder QR Factorization on Emerging GPU Architectures Using Tensor Cores](https://ieeexplore.ieee.org/document/10816084)
- [High Accuracy Matrix Computations on Neural Engines: A Study of QR Factorization and its Applications](https://dl.acm.org/doi/10.1145/3369583.3392685)
- [A Storage-Efficient WY Representation for Products of Householder Transformations](https://epubs.siam.org/doi/10.1137/0910005)

Some modification to [High Performance Householder QR Factorization on Emerging GPU Architectures Using Tensor Cores](https://ieeexplore.ieee.org/document/10816084) are introduced below to improve memory efficiency. Specifically, rather than using the traditional WY transform, the CWY Householder product representation is used.

**Note the following:**
- Einsum notations in this notebook follow [The EDGE Language: Extended General Einsums for Graph Algorithms](https://arxiv.org/abs/2404.11591).
- While the numpy code does not reflect this, the $\text{sign}$ function should follow the convention that $\text{sign}(0) = 1$ for both real and complex values.

In [ ]:
import numpy as np
import scipy
import time

We first generate random matrices that will be used to test our QR factorization implementation.

In [ ]:
randMatrices = []
for i in range(50):
    randMatrices.append(np.random.rand(150,100))

For QR decomposition, one is generally concerned with backward and orthogonality error. To ensure the correctness of our QR implementation, we will also check if matrix $R$ is actually upper triangular.

In [ ]:
def backwardError(matrix, q, r):
    """Calculates the backward error."""
    error = np.linalg.norm(matrix - np.dot(q, r), ord=2) / np.linalg.norm(matrix, ord=2)
    return error

In [ ]:
def orthogonalityError(q):
    """Calculates the orthogonality error of matrix Q."""
    error = np.linalg.norm(np.eye(q.shape[1]) - np.dot(q.T, q), ord=2)
    return error

In [ ]:
def is_upper_triangular(R, tol=1e-14):
    """Determines if matrix R is upper triangular."""
    lower_triangle_mask = np.tril_indices_from(R, k=-1)
    return np.all(np.abs(R[lower_triangle_mask]) < tol)

In [ ]:
def testQR(matrices, func):
    """Test to check if FUNC correctly computes the QR factorization for all matrices in MATRICES."""
    totalTimeCost = 0
    totalBackwardError = 0
    totalOrthogonalityError = 0
    
    for matrix in matrices:
        start = time.perf_counter()
        q, r = func(matrix)
        end = time.perf_counter()
        
        totalTimeCost += (end - start)
        
        backwardE = backwardError(matrix, q, r)
        totalBackwardError += backwardE

        orthogonalityE = orthogonalityError(q)
        totalOrthogonalityError += orthogonalityE

        isUpperTriangular = is_upper_triangular(r)

        # Tolerance levels are set to 1e-12 although the errors will not usually be this large
        if not (backwardE < np.float64(1e-12) and orthogonalityE < np.float64(1e-12) and isUpperTriangular):
            print("Test Failed")
            print("Average time:" + str(totalTimeCost/len(matrices)))
            print("Backward error: " + str(backwardE))
            print("Orthogonality error: " + str(orthogonalityE))
            print("Is upper triangular: " + str(isUpperTriangular))
            return

    print("Test Passed.")
    print("Average time:" + str(totalTimeCost/len(matrices)))
    print("Average backward error:" + str(totalBackwardError/len(matrices)))
    print("Average orthogonality error:" + str(totalOrthogonalityError/len(matrices)))
    print("Is upper triangular: " + str(isUpperTriangular))

To check if our test is functional, we first test `np.linalg.qr` and `scipy.linalg.qr`.

In [ ]:
testQR(randMatrices, np.linalg.qr)

In [ ]:
testQR(randMatrices, scipy.linalg.qr)

We start by implementing the traditional unblocked Householder QR factorization. The implementation is based on the Householder QR algorithm introduced in [Advanced Linear Algebra: Foundations to Frontiers](https://www.cs.utexas.edu/~flame/laff/alaff/chapter03-real-HQR-factorization.html).

Let $A$ be the input matrix, where $A \in \mathbb{R}^{m \times n}$ and $m \ge n$. The thin QR decomposition computes matrices $Q$ and $R$, where:
- $Q \in \mathbb{R}^{m \times n}$ (thin orthogonal matrix)
- $R \in \mathbb{R}^{n \times n}$ (upper triangular matrix).

We introduce three-tensor representations for iteration tracking.
- $R^{I,M,N}$. Let $R^{I,M,N}_{0, m, n} = A_{m,n}$.
- $Q^{I,M,N}$. Let $Q^{I,M,N}_{0, m, n} = I_{m,n}$.
- Let unspecified indices take on default values of $0$.

Index info:
- $i$: iterative rank that ranges from $[0,n-1]$.

During iteration $i$, the Householder vector and transformation are computed as follows.

Step 1: Generate the Householder vector.
$$\chi_1 = R^{I,M,N}_{i,i,i}$$
$$\chi_2 = \sqrt{R^{I,M,N}_{i, m:m \ge i+1, i}R^{I,M,N}_{i, m:m \ge i+1, i}}$$
$$\alpha = \sqrt{\chi_1^2 + \chi_2^2}$$
$$\nu = \chi_1 + \text{sign}(\chi_1)\alpha$$
$$V_{i,m:m \ge i+1} = \frac{R^{I,M,N}_{i, m:m \ge i+1, i}}{\nu}$$
$$\chi_2 = \frac{\chi_2}{\text{abs}(\nu)}$$
$$\tau = \frac{1 + \chi_2^2}{2}$$

Step 2: Update the $Q$ and $R$ matrices.
$$
U_{i,m} =
\begin{cases} 
0, & \text{if } m < i, \\
1, & \text{if } m = i, \\
V_{i,m}, & \text{if } m > i
\end{cases}
$$

$$P_{i,m,n} = \frac{1}{\tau} U_{i,m}U^{I,M}_{i,n}$$
$$G_{i,m,n} = P^{I,M,N}_{i,m,k}R^{I,M,N}_{i, k, n}$$
$$R_{i+1,m,n} = R_{i,m,n} - G_{i,m,n}$$

$$T_{i,m,n} = Q^{I,M,N}_{i,m,k}P^{I,M,N}_{i,k,n}$$
$$Q^{I,M,N}_{i+1,m,n} = Q^{I,M,N}_{i,m,n} - T_{i,m,n}$$

The final output of the algorithm will be:
- $Q = Q^{I,M,N}_{N+1, m,n}$
- $R = R^{I,M,N}_{N+1, m,n}$

Because $U$ contains $0$ for all $m<i$
$$P=\begin{bmatrix}
        0 & 0 \\
        0 & P_4
    \end{bmatrix},$$ so $P_4$ is applied to update $R$ and $Q$ so we can update $Q$ and $R$ as follows.
$$PR = \begin{bmatrix}
        0 & 0 \\
        0 & P_4
    \end{bmatrix}\begin{bmatrix}
        R_1 & R_2 \\
        0 & R_4
    \end{bmatrix} = \begin{bmatrix}
        0 &  0\\
        0 & P_4R_4
    \end{bmatrix}$$
$$QP = \begin{bmatrix}
        Q_1& Q_2
    \end{bmatrix}\begin{bmatrix}
        0 & 0 \\
        0 & P_4
    \end{bmatrix} = \begin{bmatrix}
        0 & Q_2P_4
    \end{bmatrix}$$

Thus, the program can be made efficient by avoiding ineffectual operations.

Notice that in the cascade of Einsums here, the Householder matrix is not explicitly form but applied implicitly. In the `unblockedHouseholderQR` function below, the Householder matrix will still be explicitly formed but implicit update will be used from `blockedHouseholderQR` onwards for efficiency.

In [ ]:
def householderReflector(x):
    """Computes the Householder vector given an input vector."""
    chi_1 = x[0]
    chi_2 = np.linalg.norm(x[1:], ord=2) # np.sqrt(np.einsum('i,i->', x[1:], x[1:]))

    alpha = np.sqrt(chi_1**2 + chi_2**2) # 2-norm of the input vector

    # Construct the Householder vector below the first element
    v = chi_1 + np.sign(chi_1) * alpha
    u_2 = x[1:] / v
    chi_2 /= np.abs(v)
    
    # Compute the tau constant
    tau = 0.5 * (1 + chi_2 ** 2)
    
    return u_2, tau

def unblockedHouseholderQR(A):
    """Computes the unblocked Householder QR factorization for input matrix A."""
    R = np.copy(A)
    m, n = A.shape
    Q = np.eye(m)

    for i in range(n):
        # Construct the Householder vector and its corresponding tau constant
        u_2, tau = householderReflector(R[i:,i])
        u = np.concatenate(([1], u_2))
        
        # Build the Householder matrix H
        H = np.eye(u.shape[0]) - 1/tau * np.outer(u,u) # np.einsum('i,j->ij', u, u)
        
        # Apply the Householder transformation to Q and R matrices
        R[i:,i:] = H @ R[i:,i:] # np.einsum('ij,jk->ik', H, R[i:,i:])
        Q[:,i:] = Q[:,i:] @ H # np.einsum('ij,jk->ik', Q[:,i:], H)
    
    Q = Q[:,:n]
    R = R[:n,:]
    return Q, R

For matrices with a shape larger than 100 in either dimension, the `unblockedHouseholderQR` can be really slow.

In [ ]:
testQR(randMatrices, unblockedHouseholderQR)

Next, we will implement blocked Householder QR via compact WY transform. The implementation is based on the paper [A Storage-Efficient WY Representation for Products of Householder Transformations](https://epubs.siam.org/doi/10.1137/0910005).

The compact WY (CWY) transform involves matrix $Y$ and $T$. For an input panel matrix of shape $A \in \mathbb{R}^{m \times n}$, $Y \in \mathbb{R}^{m \times n}$  and $T \in \mathbb{R}^{n \times n}$. By construction, $Y$ is lower triangular, $T$ is upper triangular, and $Q=I-YTY^T \in \mathbb{R}^{m \times m}$. Compared to the standard WY transform, as documented in [The WY Representation for Products of Householder Matrices](https://epubs.siam.org/doi/abs/10.1137/0908009), CWY is more memory efficient. The WY representation encodes Householder products using two matrices, $W \in \mathbb{R}^{m \times n}$ and $Y \in \mathbb{R}^{m \times n}$. In the case where $m \gg n$, which is usually the case for when performing panel factorization, the compact WY representation is generally more preferable. In general the CWY representation is also more preferable as many linear algebra packets also implements QR decomposition using CWY transform.

We will proceed to explain the CWY representation briefly.
Let $Q$ be the product of $n$ Householder matrices. We define $Q_+$ as
$$Q_+ = QH_{n+1},$$
where $H_{n+1}$ is the $[n+1]$-th Householder matrix. Recall that the Householder reflector is defined as $H_{n+1} = I-\frac{u_{n+1}u_{n+1}^T}{\tau_{n+1}}$. where $u_{n+1}$ is the Householder reflector. For brevity, let $u=u_{n+1}$ and $\tau = \tau_{n+1}$ below.

We want to show that
$$Q_+ = QH_{n+1} = (I-YTY^T)H_{n+1} = I-Y_+T_+Y_+^T,$$
where
$$Y_+ = [Y, u] \in \mathbb{R}^{m \times \{n + 1\}}$$
and 
$$T_+ = \left[
\begin{array}{c c}
T & z \\
0 & \rho
\end{array}
\right] \in \mathbb{R}^{\{n + 1\} \times \{n + 1\}}$$
with $\rho = \frac{1}{\tau}$, and $z = -\rho TY^Tu$.

To prove this, see that,
$$\begin{align*}
    I - Y_+ T_+ Y_+^T &= I - 
    \begin{bmatrix}
        Y & u
    \end{bmatrix}
    \begin{bmatrix}
        T & z \\
        0 & \rho
    \end{bmatrix}
    \begin{bmatrix}
        Y^T \\
        u^T
    \end{bmatrix} \\[8pt]
    &= I - 
    \begin{bmatrix}
        Y & u
    \end{bmatrix}
    \begin{bmatrix}
        TY^T + zu^T \\
        \rho u^T
    \end{bmatrix} \\[8pt]
    &= I - YTY^T - Yzu^T - \rho uu^T \\
    &= I - YTY^T + \frac{1}{\tau} YTY^Tuu^T - \frac{1}{\tau}uu^T.
\end{align*}$$

Note that
$$Q_+ = QH_{n+1} = (I-YTY^T)(I-\frac{1}{\tau}uu^T)=I-YTY^T+\frac{1}{\tau}YTY^Tuu^T-\frac{1}{\tau}uu^T.$$

Thus, we have proven that the CWY representation is valid by induction as the base case is trivially true given that the initial construction for the $Y$ and $T$ matrices is just:
$$Y = [u_1]$$
and
$$T = \begin{bmatrix}
            \frac{1}{\tau_1}
        \end{bmatrix},$$
where $u_1$ and $\tau_1$ can be used to form $H_1$, the first Householder reflector.

If a complete $Q$ is necessary, $Q=I-YTY^T \in \mathbb{R}^{m \times m}$. If a thin $Q$ suffices, then $Q = I-YTY_1^T \in \mathbb{R}^{m \times n}$, where $Y_1$ is the upper square section of $Y$. This definition of $Y_1$ will continue to be used throughout this notebook.

Rather than using purely sequential updates, as shown above, which mainly involves level 2 BLAS operations, we will attempt to increase the use of matrix multiplications.

After computing $Y_{new}$ and $T_{new}$ for a panel in `panelQR`, we now have
$$T=\begin{bmatrix}
        T_{\text{old}} & 0 \\
        0 & T_{\text{new}}
    \end{bmatrix}.$$
$$Y=\begin{bmatrix}
        Y_{\text{old}} & Y_{\text{new}}
    \end{bmatrix}.$$

We will need to update $T$ to ensure global correctness. We consider
$$T=\begin{bmatrix}
        T_{\text{old}} & T_{\text{cross}} \\
        0 & T_{\text{new}}
    \end{bmatrix}.$$

Our goal is to determine $T_{\text{cross}}$.
From the recursive definition of compact WY transformation along with zero padding to ensure matching matrix dimensions, we have:
$$\begin{align}
Q_{\text{old}}Q_{\text{new}}&=(I-Y_{\text{old}}T_{\text{old}}Y^T_{\text{old}})(I-Y_{\text{new}}T_{\text{new}}Y^T_{\text{new}})\\
&= I - Y_{\text{new}}T_{\text{new}}Y^T_{\text{new}} - Y_{\text{old}}T_{\text{old}}Y^T_{\text{old}} + Y_{\text{old}}T_{\text{old}}Y^T_{\text{old}}Y_{\text{new}}T_{\text{new}}Y^T_{\text{new}}\\
&= I-YTY^T\\
&= I - Y_{\text{new}}T_{\text{new}}Y^T_{\text{new}} - Y_{\text{old}}T_{\text{old}}Y^T_{\text{old}} - Y_{\text{old}}T_{\text{cross}}Y^T_{\text{new}}\\
\end{align}
$$

We observe that the definition such that
$$T_{\text{cross}}=-T_{\text{old}}Y^T_{\text{old}}Y_{\text{new}}T_{\text{new}}$$
captures the interaction between old and new reflectors, ensuring that the blocked structure maintains the recursive formulation of the compact WY representation. This concludes how blocked updates can be generalized to ensure global correctness.

We will attempt to represent the blocked Householder QR algorithms in terms of Einsum cascades.

Initially:
- $R^{I,M,N}_{0, m, n} = A_{m,n}$.
- $Y^{I,M,N}$
- $T^{J,K,N,P}_{0,0,n,p} = 0^{n \times n}$

Iteratively, `Y` and `T` will be updated.

Index info:
- We want to build matrix $Y$ and $T$ while updating $R$ along rank $N$ iteratively. For `panelQR` we want to partition the iterative $N$ into an $N1$ and an $N0$.
- Let the iterative $N1$ be $K$ and the iterative $N0$ be $J$.

$$\text{Furthermore, let the panel be } \tilde{A}^{J,K,M,N}_{0,k,m,n}=A^{M,N}_{m:m\ge k * J, n:k*J \le n < (k+1)J}$$
**Please remember that the $\tilde{A} \ne A$. $\tilde{A}$ is a panel of the input matrix $A$.**

Let $V$ and $\tau$ be the values returned by calling `householderReflector`. The definition of $V$ and $\tau$ as well as the `householderReflector` function follows the the Einsum shown in the unblocked Householder QR section. Recall that `householderReflector` returns elements following the first element in the Householder vector and the tau constant.

Let's consider a fixed value for $k$ and $j$. We will also omit $k$ and $j$ iterative ranks for brevity.

Step 1: Construct the Householder vector
$$
U_{m} =
\begin{cases}
0, & \text{if } m < kJ + j, \\
1, & \text{if } m = kJ + j, \\
V_{m-1}, & \text{if } m > kJ + j
\end{cases}
$$

Step 2: Update $Y$

$$Y^{M,N}_{m,kJ + j} = U_{m}$$

Step 3: Update $\tilde{A}$
$$\rho = \frac{1}{\tau}$$
$$P_{m,n} = \rho U_{m}U^{M}_{n}$$
$$G_{m,n} = 
\begin{cases}
P^{M,N}_{m,l}\tilde{A}^{M,N}_{l,n}, & \text{if } (n \ge kJ+j),\\
0, & \text{else}
\end{cases}$$
$$\text{Update $j$ to $j+1$: }\tilde{A}_{m,n} = \tilde{A}_{m,n} - G_{m,n}$$

Step 4: Update $T$

For this step, the bounds are defined as:
- $m \ge kJ + j$
- $kJ \le n,p < kJ+j$
- Unspecified elements take the default value of $0$.
$$B_{n} = Y_{m,n}U_{m}$$
$$Z_{n} = -\rho T_{n,p}B^{N}_{p}$$
$$T_{n,p} =
\begin{cases}
\rho, & \text{if } (n = p = kJ+j),\\
Z_{n}, & \text{if } (p = kJ+j) \wedge (n < p),\\
T_{n,p}, & \text{else}
\end{cases}$$

In [ ]:
def householderReflector(x):
    """Computes the Householder vector given an input vector."""
    chi_1 = x[0]
    chi_2 = np.linalg.norm(x[1:], ord=2) # np.sqrt(np.einsum('i,i->', x[1:], x[1:]))

    alpha = np.sqrt(chi_1**2 + chi_2**2) # 2-norm of the input vector++++

    # Construct the Householder vector below the first element
    v = chi_1 + np.sign(chi_1) * alpha
    u_2 = x[1:] / v
    chi_2 /= np.abs(v)
    
    # Compute the tau constant
    tau = 0.5 * (1 + chi_2 ** 2)
    
    return u_2, tau


def panelQR(A, Y, T, k):
    """Update the Y,T matrices for a block of matrix A."""
    m, n = A.shape
    
    for i in range(n):
        idx = k + i  # Relative index in Y and T

        # Compute Householder vector and tau
        Y[idx+1:, idx], tau = householderReflector(A[i:, i])
        u = Y[idx:, idx]

        # Apply Householder transformation to reduce panel A
        rho = 1 / tau
        A[i:, i:] -= rho * np.outer(u, np.dot(u, A[i:, i:])) # np.einsum('i,j->ij', u, np.einsum('i,ij->j', u, A[i:, i:]))

        # Update the T matrix
        T[idx, idx] = rho
        if i > 0:
            z = T[k:idx, k:idx] @ Y[idx:, k:idx].T @ u
            T[k:idx, idx] = -rho * z

We will continue to discuss the `blockedHouseholderQR` function. The convention for variables follow the previous section on `panelQR`.

In this function, we will call `panelQR` on $\tilde{A}$ to compute the Householder QR of the panel and generalize the $Y$ and $T$ matrices so they are correct globally.

**m and n are not restricted to a certain range of values here. Furthermore, we will only omit the $J$ iterative rank here. Rank $K$ is needed.**

To update the global $Y$ and $T$ matrices as well as the trailing matrix, we compute:

$$C_{k,m,n}=Y^{K,M,N}_{q: q < k, l:l \ge kJ,m}Y^{K,M,N}_{k, l,n}$$
$$D_{k,m,p}=C_{k,m,n}T^{K,N,P}_{k,n:kJ\le n < (k+1)J,p}$$
$${E}_{k,n,m}=-T^{K,N,P}_{q:q<k, n:n<kJ, p}D^{K,M,P}_{k,p,m}$$
$$T_{k,n,p}=E^{K,N,M}_{k,n,p}$$
$$F_{q,l,n}=
\begin{cases} 
Y^{K,M,N}_{k,p:p\ge kJ,l}R^{K,M,N}_{q,p,n}, & \text{if } (kJ \le l < (k+1)J) \wedge (q\ge k+1), \\
0, & \text{else}
\end{cases}
$$
$$W_{q,m,n} = T^{K,N,P}_{k,l,m}F_{q:q\ge k+1,l,n}$$
$$\text{Update $k$ to $k+1$: }R_{q,m,n} = R_{q,m,n} - Y^{K,M,N}_{k,m,t}W^{K,M,N}_{q:q\ge k+1,t,n}$$

In [ ]:
def blockedHouseholderQR(A, block_size=30, complete=False, householder_repr = False):
    """Computes a blocked Householder QR factorization using Compact WY representation."""
    m, n = A.shape
    R = np.copy(A)
    Y = np.eye(m, n)
    T = np.zeros((n, n))
    
    for k in range(0, n, block_size):
        bk = min(block_size, n - k)  # Block size

        # Compute a panel of the input matrix
        panelQR(R[k:, k:k+bk], Y, T, k)

        if k != 0:
            # Compute T_cross
            T_cross = -T[:k, :k] @ (Y[k:, :k].T @ Y[k:, k:k+bk]) @ T[k:k+bk, k:k+bk] # -np.einsum("ij,jk,kl,lm->im", T[:k, :k], Y[k:, :k].T, Y[k:, k:k+bk], T_panel)
            # Update the global T matrix
            T[:k, k:k+bk] = T_cross

        # Trailing matrix update
        W = T[k:k+bk, k:k+bk].T @ Y[k:, k:k+bk].T @ R[k:, k+bk:] # np.einsum('ji,jk->ik', T[k:k+bk, k:k+bk], np.einsum('ji,jk->ik', Y[k:, k:k+bk], R[k:, k+bk:]))
        R[k:, k+bk:] -= np.dot(Y[k:, k:k+bk], W) # np.einsum('ij,jk->ik', Y[k:, k:k+bk], W)

    if householder_repr:
        return Y, T, R
    elif complete:
        return np.eye(m, m) - np.dot(Y, np.dot(T, Y.T)), R # np.einsum('ij,jk,kl->il', Y, T, Y.T), R
    else:
        return np.eye(m, n) - np.dot(Y, np.dot(T, Y[:n, :].T)), R[:n, :] # np.einsum('ij,jk,kl->il', Y, T, Y[:n, :].T), R[:n, :]

In [ ]:
testQR(randMatrices, blockedHouseholderQR)

Now, let us introduced tall-skinny QR(TSQR). This section is based on [Communication-optimal Parallel and Sequential QR and LU Factorizations](https://epubs.siam.org/doi/10.1137/080731992).

Idea: tile along the rows of a tall matrix so we only need to process matrices that are approximately square one at a time. This improves the use of tensor engines and cache.
$$
\begin{align*}
\begin{bmatrix}
    A_{1}\\
    A_{2}\\
    A_{3}\\
    A_{4}
\end{bmatrix} 
&= \begin{bmatrix}
    Q_{11}R_1\\
    Q_{12}R_2\\
    Q_{13}R_3\\
    Q_{14}R_4
\end{bmatrix} 
= \begin{bmatrix}
    Q_{11}\\
    & Q_{12}\\
    & & Q_{13}\\
    & & & Q_{14}
\end{bmatrix} 
\begin{bmatrix}
    R_1\\
    R_2\\
    R_3\\
    R_4
\end{bmatrix} \\
&= \begin{bmatrix}
    Q_{11}\\
    & Q_{12}\\
    & & Q_{13}\\
    & & & Q_{14}
\end{bmatrix} \begin{bmatrix}
    Q_{21}\\
    Q_{22}\\
    Q_{23}\\
    Q_{24}
\end{bmatrix} R \\
&= \begin{bmatrix}
    Q_{11}Q_{21}\\
    Q_{12}Q_{22}\\
    Q_{13}Q_{23}\\
    Q_{14}Q_{24}
\end{bmatrix} R = QR
\end{align*}$$

Matrix $R$ can be computed recursively if it is still tall. The base case of the recursive call can be when the matrix is reduced enough so it fits in the Tensor core. Thus, tiling along the rows can be done multiple times to reduce matrix height during panelQR. This process produces a thin QR factorization and guarentees a globally orthonormal $Q$ matrix as it is a product of multiple orthonormal $Q$ factors.

Notice that the $Q$ factors do not need to be explicitly contracted if an explicit $Q$ matrix is not needed. However, for our purpose, an explicit $Q$ matrix needs to be generated. The reason as to why an explicit $Q$ is necessary will be revealed later.

`tsqr` involves straightforward recursive decomposition and matrix multiplication. The Einsum cascade is simple so we will skip showing `tsqr` in terms of Einsum cascades here.

In [ ]:
def tsqr(A):
    """Performs TSQR."""
    m, n = A.shape
    if m > n * 2:
        # Tiles the input matrix horizontally by two
        Q_11, R_1 = tsqr(A[:m // 2, :])
        Q_12, R_2 = tsqr(A[m // 2:, :])

        # Factorize R recursively to ensure global orthogonality
        R = np.vstack((R_1, R_2))
        Q_R, R_R = tsqr(R)

        # Update the Q matrix
        Q_1 = Q_11 @ Q_R[:Q_11.shape[1], :] # np.einsum("ij,jk->ik", Q_11, Q_R[:Q_11.shape[1], :])
        Q_2 = Q_12 @ Q_R[Q_11.shape[1]:, :] # np.einsum("ij,jk->ik", Q_12, Q_R[Q_11.shape[1]:, :])
    
        return np.vstack((Q_1, Q_2)), R_R

    Q, R = blockedHouseholderQR(A)
    return Q, R

In [ ]:
testQR(randMatrices, tsqr)

Notice if an input matrix is large, then a block of matrix `blockedHouseholderQR` is tall and skinny. This makes TSQR, a type of communication avoiding QR factorization(CAQR) algorithm, a good fit when factoring blocks of matrices. However, TSQR produces implicit tree-structed $Q$ factors as shown in [Communication-optimal Parallel and Sequential QR and LU Factorizations](https://epubs.siam.org/doi/10.1137/080731992). Managing the implicit $Q$ matrix can be complicated so an explicit $Q$ is computed in `TSQR`.

Another problem arises when we consider trailing matrix update. While theoretically $A=QR$, updating the trailing matrix using an explicit $Q$ is numerically unstable and introduces orthogonality loss. Thus, we will need to reconstruct the CWY representation of the Householder product from the result of TSQR.

The process of reconstruction is the most important part of this notebook, and it will be explained briefly. The reconstruction of the CWY representation from the result of TSQR is detailed more clearly in [Reconstructing Householder Vectors from Tall-Skinny QR](https://ieeexplore.ieee.org/document/6877344).

Let $A$ be the matrix of interest for decomposition. The key idea in the paper is that LU decomposition can be applied to $A-R$. Recall that the CWY representation shows that it is possible to express $A$ in terms of 
$$A = (I-YTY^T)R$$
so
$$A - R = Y(-TY^TR).$$
Since $Y$ is lower triangular while $T$, $Y^T$, and $R$ are upper triangular, this equation gives a unique LU decomposition of $A-R$. However, calling $\text{LU}(A-R)$ is not numerically stable. Consider a low rank matrix $A$ (there are linearly dependent columns), the non-uniqueness of QR decomposition means that $(Q_{TSQR}, R_{TSQR})$ and $(Q_{Hh}, R_{Hh})$ may be different factor pairs. Then, performing LU decomposition on $A-R_{TSQR}$ in an attempt to recover Householder vectors that make up $Q_{Hh}$ is hopeless. Thus, for ill conditioned matrices, this method can be unstable.

We will now shift our focus to reconstructing the Householder products of an orthonormal matrix $A$. Let $S$ be the sign matrix to the diagonal elements of $Q$. If $Q$ is tall, $S$ will just cover the upper square part of $Q$. Notice that QR decomposition on an orthonormal matrix $A$ is almost unique. We are only left with sign ambiguities. Then, the key idea is to set $R=S$, where $A=QS.$ Since $S$ is a diagonal sign matrix corresponding to the sign choices made inside the Householder-QR algorithm, performing an LU decomposition on $A-S$ is equivalent to performing the Householder-QR.

Thus, to reconstruct the Householder product for a sqaure orthogonal matrix $Q$, we will:
Perform $\text{LU}(Q-S)$, where $S$ is the sign matrix of the square block of $Q$. We will get $Q-S=LU$ in this step. $L$ is exactly the lower triangular Householder vector matrix $Y$.

To solve for $T$, we can then compute $T = -US^{-1}Y^{-T}.$ Since $Y_1$ is lower triangular, we can solve for $Y^{-T}$ using triangular solve with multiple right hand sides.

Finally, the algorithm will output the decomposition $A=(I-YTY^T)(SR)$. This process essentially finds $A=QR$ through TSQR, reconstructs $Q=(I-YTY^T)(S)$ via LU decomposition, then outputs the final decomposition in the Householder form $A=(I-YTY^T)(SR)$.

The following equation summarizes the `tsqr_hr` function that works not just for an orthogonal square matrix but for any general matrix $A$.

Given a tall matrix $A \in \mathbb{R}^{m \times b}$ with $m \gg b$, the goal is to compute the QR decomposition using TSQR and reconstruct the Householder product in Compact WY form.

Several points and steps
1. Let us first separate $A$, 
$$A =\begin{bmatrix}A_1 \\ A_2\end{bmatrix}, \quad A_1 \in \mathbb{R}^{b \times b}, \quad A_2 \in \mathbb{R}^{(m-b) \times b}.$$
2. Compute QR for the lower block: $\text{TSQR}(A_2) \rightarrow A_2 = Q_1 R_1.$ Now we have
$$A = \begin{bmatrix}A_1 \\ Q_1 R_1\end{bmatrix}$$
Notice that
$$A = \begin{bmatrix}A_1 \\ Q_1 R_1\end{bmatrix} =  \begin{bmatrix}I_b &  \\ & Q_1\end{bmatrix} \begin{bmatrix}A_1 \\ R_1\end{bmatrix} = \begin{bmatrix}I_b &  \\ & Q_1\end{bmatrix} Q_2 \tilde R = \begin{bmatrix}I_b &  \\ & Q_1\end{bmatrix} \begin{bmatrix} Q_2^{(1)}  \\ Q_2^{(2)} \end{bmatrix} R = \begin{bmatrix}Q_2^{(1)} \\ Q_1Q_2^{(2)} \end{bmatrix} R = QR,$$
where $Q_2^{(1)}$ and $Q_2^{(2)}$ are the upper and lower square matrix in $Q_2$.
4. Stack the upper part of $A$ with $R_1$ and perform QR again: $$\tilde{A} = \begin{bmatrix} A_1 \\ R_1 \end{bmatrix}, \quad \text{TSQR}(\tilde{A}) \rightarrow \tilde{A} = Q_2 \tilde{R}$$
5. `modifiedLU` is called on $Q_2^{(1)}$ to reconstruct the Householder transform. So we now get $L = Y_1$ as well as $U$ and $S$.
6. $R = S\tilde{R}$
7. Compute the correction factor for the lower block of $Y$: $$W = Q_2^{(2)} U^{-1}$$
8. Compute the lower part of $Y$: $$Y_{\text{lower}} = Q_1 W$$
9. Construct the full Householder matrix: $$Y =
   \begin{bmatrix}
   Y_1 \\ Y_{\text{lower}}
   \end{bmatrix}$$
10. Compute the Compact WY representation: $$T = -U S Y_1^{-T}$$

Let us attempt to show the algorithm described above in terms of Einsum cascades.

We start by describing the `modifiedLU` function in terms of Einsums cascades. The function takes an orthogonal matrix $Q^{I,M,B} \in \mathbb{R}^{m \times b}$ as input.

Index info:
- Let $i$ take on the value of `i` in `for i in range(b)`. 

First, we initialize the following matrices:
- $S_{n,b}$.
- $U_{n,b}$.
- $L^{I,M,B}_{0,m,b} = I_{m \times b}$.

$$S^{N,B}_{i,i} = -\text{sign}(Q^{I,M,B}_{i,i,i})$$
$$U^{N,B}_{i,i} = Q^{I,M,B}_{i,i,i} - S^{N,B}_{i,i}$$
$$U^{N,B}_{i,b:b \ge i+1} = Q^{I,M,B}_{i,i,b \ge i+1}$$
$$L^{M,B}_{m:m\ge i+1,i}=Q^{I,M,B}_{i,m:m\ge i+1,i}/U^{N,B}_{i,i}$$
$$
Q_{i+1,m,b}=
\begin{cases}
Q_{i,m,b}-L^{M,B}_{m,i}U^{N,B}_{i,b}, & \text{if } (m \ge i + 1) \wedge (b \ge i + 1), \\
Q_{i,m,b}, & \text{else}
\end{cases}$$

In [ ]:
def modifiedLU(Q):
    """Performs LU factorization without pivoting on Q"""
    m, b = Q.shape
    S = np.zeros((b, b))
    U = np.zeros((b, b))
    L = np.eye(m, b)
    
    for i in range(b):
        S[i, i] = -np.sign(Q[i, i])
        U[i, i] = Q[i, i] - S[i,i]
        U[i, i+1:b] = Q[i, i+1:b]
        L[i+1:m, i] = Q[i+1:m, i] / U[i, i]
        Q[i+1:m, i+1:b] -= np.outer(L[i+1:m, i], U[i, i+1:b]) # np.einsum("j,k->jk", L[i+1:m, i], U[i, i+1:b])
    
    return L, U, S

To reconstruct the Householder representation, `tsqr_hr` will be used. The function takes any matrix $A^{M,B} \in \mathbb{R}^{m \times b},$ performs TSQR or Householder QR, and returns the Householder representation of the transformation. The Einsum of `tsqr_hr` is given below:

`if M < 2 * b:`, the input matrix is not tall so `blockedHouseholderQR` is used.

`else`:

`tsqr` is called on $A^*,$ where
$${A^*}^{M,B}_{m,b}=A_{m: \, m \ge B, \, b}$$
producing
$${A^*}_{m,b} = Q1_{m,p} R1_{p,b}.$$
Then, `tsqr` is called on $\tilde{A}$. $\tilde{A}$ is defined as
$$\tilde{A}_{m,b} = 
\begin{cases}
A_{m, b:b}
R1^{P,B}_{m, b}
\end{cases}$$
After calling `tsqr`, we factorize $\tilde{A}$ into
$$\tilde{A}_{m,b} = Q2_{m,l} \tilde{R}_{l,b}.$$
Using `modifiedLU` on $Q2^{M,L}_{m: m < L, \, l}$, we get the decomposition along with the sign matrix:
- $S^{N,Q}$
- $U^{N,Q}$
- $Y1^{M,Q}$

$$R_{n,b} = S^{N,Q}_{n,l} \tilde{R}_{l,b}$$

To find $U^{-1}$, we use triangular solve. Let $U^{-1} = {U^{(-1)}}^{L,N}.$

$$W^{M,N}_{m,n} = Q2^{M,L}_{m:m\ge L, l}{U^{(-1)}}_{l,n}$$
$${Y^{(\text{lower})}}_{m,q}=Q1_{m,p}W^{M,N}_{p,q}$$
$$Y_{m,q}=Y1_{m,q}+{Y^{(\text{lower})}}_{m,q}$$

Since $Y1^{M,Q}$ is lower triangular, we can find $Y1^{-T}$ using triangular solve. Let 
$Y1^{-T}={Y1^{(-T)}}^{Q,M}$
$$T_{p,m} = - U^{N,Q}_{p,n} S_{n,q}{Y1^{(-T)}}_{q,m}$$

The Householder representation is reconstructed and encoded in $Y$ and $T$.

The final output of `tsqr_hr` includes
- $T^{P,M}$
- $Y^{M,Q}$
- $R^{N,B}$

The Einsums for `triangular_matrix_inversion` is also given below. This function employs the standard back substitution algorithm.

Consider the following definitions:
- The input upper triangular matrix is $U \in \mathbb{R}^{M,N}$. Note that $M=N$.
- $U^{-1}$ will be represented using $P$, where $P$ must also be upper triangular.

Let $j$ iterate from $0$ to $N-1$
$$P^{M,N}_{j,j} = 1/U^{M,N}_{j,j}$$

Let $i$ iterate from $j-1$ to $0$.
$$\alpha_{i,j} = -U^{M,N}_{i,n:i+1 \le n \le j}P^{M,N}_{n,j}$$
$$P^{M,N}_{i,j}=\alpha_{i,j}/U^{M,N}_{i,i}$$

**Note that `ModifiedLU` and `triangular_inversion` need not be blocked as they are small matrices. Blocking them may only increase overhead, leading to a significant increase in flops without additional benefits.**

In [ ]:
def triangular_matrix_inversion(U):
    """Computes the inverse of an upper triangular matrix U using back-substitution."""
    n = U.shape[0]
    U_inv = np.zeros((n,n))

    for j in range(n):
        U_inv[j, j] = 1.0 / U[j, j]
        for i in range(j - 1, -1, -1):
            U_inv[i, j] = -U[i, i+1:j+1] @ U_inv[i+1:j+1, j] / U[i, i]
    return U_inv

def tsqr_hr(A, Y, T, k):
    """Performs TSQR and reconstructs the Householder representation"""
    m, b = A.shape

    # The standard tsqr_hr only works for tall matrices such that the (number of rows) > (number of columns * 2)
    if m < 2 * b:
        Y[:, :], T[:b, :b], A[:b,:] = blockedHouseholderQR(A, householder_repr = True)
    Q1, R1 = tsqr(A[b:, :])
    Q2, A[:b,:] = tsqr(np.vstack([A[:b, :], R1]))
    Y[:b, :b], U, S = modifiedLU(Q2[:b, :])
    
    U_inv = triangular_matrix_inversion(U)
    # U_inv = np.linalg.inv(U)
    
    W = Q2[b:, :] @ U_inv # np.einsum("ij,jk->ik", Q2[b:2*b, :], U_inv)
    Y[b:, :] = Q1 @ W # np.einsum("ij,jk->ik", Q1, W)

    Y1_T_inv = triangular_matrix_inversion(Y[:b, :b].T)
    # Y1_T_inv = np.linalg.inv(Y[:b, :b].T)
    
    T[:, :] = -U @ S @ Y1_T_inv # -np.einsum("ij,jk,kl->il", U, S, Y1_T_inv)
    A[:b,:] = S @ A[:b,:] # np.einsum("ij,jk->ik", S, R_tilde)

To combine TSQR with our blocked Householder algorithm, we just need to call `tsqr_hr` for panel factorization and update the global matrix $Y$ and $T$.

`BH_TSQR` share the same set of Einsum cascades as the the one shown in `blockedHouseholderQR` with slight changes to indexing because `panelQR` and `tsqr_hr` both handle the same task (construct CWY representation of the Householder product for a panel of input `A`).

In [ ]:
def BH_TSQR(A, block_size=25):
    """Performs QR factorization based on blocked Householder and TSQR."""
    m, n = A.shape

    # tsqr_hr only works for tall matrices
    if m < block_size * 2:
        return blockedHouseholderQR(A)
    
    R = np.copy(A)
    Y = np.zeros((m, n))
    T = np.zeros((n, n))
    
    for k in range(0, n, block_size):
        # Handle edge case
        bk = min(block_size, n - k)
        
        # Compute a panel of the input matrix
        tsqr_hr(R[k:, k:k+bk], Y[k:, k:k+bk], T[k:k+bk, k:k+bk], k)
        
        if k != 0:
            # Compute T_cross
            T_cross = -T[:k, :k] @ (Y[k:, :k].T @ Y[k:, k:k+bk]) @ T[k:k+bk, k:k+bk] # -np.einsum("ij,jk,kl,lm->im", T[:k, :k], Y[k:, :k].T, Y[k:, k:k+bk], T_panel)
            # Update the global T matrix
            T[:k, k:k+bk] = T_cross

        # Trailing matrix update
        W = T[k:k+bk, k:k+bk].T @ Y[k:, k:k+bk].T @ R[k:, k+bk:] # np.einsum('ji,jk->ik', T[k:k+bk, k:k+bk], np.einsum('ji,jk->ik', Y[k:, k:k+bk], R[k:, k+bk:]))
        R[k:, k+bk:] -= np.dot(Y[k:, k:k+bk], W) # np.einsum('ij,jk->ik', Y[k:, k:k+bk], W)

    Q = np.eye(m, n) - Y @ T @ Y[:n, :].T # np.einsum("ij,jk,kl->il", Y, T, Y[:n, :].T)

    return Q, np.triu(R[:n, :])

In [ ]:
testQR(randMatrices, BH_TSQR)

We will perform some tests to compare our implementation with Numpy's implementation. Please note that our implementation is poorly optimized as Python does not provide direct access to more low level control of the machine.

In [ ]:
def test_qr(matrices, func):
    """Test to check if FUNC correctly computes the QR factorization for matrices in MATRICES."""
    totalTimeCost = 0
    totalBackwardError = 0
    totalOrthogonalityError = 0
    
    for matrix in matrices:
        start = time.perf_counter()
        q, r = func(matrix)
        end = time.perf_counter()
        
        totalTimeCost += (end - start)
        
        backwardE = backwardError(matrix, q, r)
        totalBackwardError += backwardE

        orthogonalityE = orthogonalityError(q)
        totalOrthogonalityError += orthogonalityE

        isUpperTriangular = is_upper_triangular(r)

        # Tolerance levels are set to 1e-12 although the errors will not usually be this large
        if not (backwardE < np.float64(1e-12) and orthogonalityE < np.float64(1e-12) and isUpperTriangular):
            print("Test Failed")
            return

    print("Time taken:" + str(totalTimeCost/len(matrices)))

In [ ]:
for i in range(40):
    randMatrix = [np.random.rand(10000 + 1000 * i, 1000)]
    print(randMatrix[0].shape)
    test_qr(randMatrix, np.linalg.qr)
    test_qr(randMatrix, BH_TSQR)
    print()